# Project 4 - CLUSTERING
UID: 404749568 , 904729596 , 704774938

In [1]:
import re
import math
import string
import operator
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt; plt.rcdefaults()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import text
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from statsmodels.discrete.discrete_model import Logit
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from scipy.sparse import linalg
from sklearn.decomposition import NMF

## Question 5

In [3]:
twenty_train = fetch_20newsgroups(subset = 'all', shuffle=True,random_state=42)
new_target = twenty_train.target[:]

In [4]:
def stemming_tokenizer(text):
    stemmer = SnowballStemmer("english")
    interim = text 
    interim = "".join([a for a in interim if a not in set (string.punctuation)])
    interim = re.sub('[,.-:/()?{}*$#&]',' ',interim)
    interim = "".join(b for b in interim if ord(b) < 128)
    words = interim.split()
    stemmed = [stemmer.stem(item) for item in words]
    return stemmed
final_vect = CountVectorizer(analyzer = 'word' , tokenizer = stemming_tokenizer,stop_words = 'english',min_df = 5,max_df=0.8)
final_counts = final_vect.fit_transform(twenty_train.data)
print("Dimensions of Numerical feature vector: "+ str(final_counts.shape))
print ("Number of terms Extracted: " + str(final_counts.shape[1]))

#TFIDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(final_counts)
print("Dimensions of TF-IDF vector: "+ str(X_train_tfidf.shape))


Dimensions of Numerical feature vector: (18846, 24283)
Number of terms Extracted: 24283
Dimensions of TF-IDF vector: (18846, 24283)


In [5]:
U, s, V = linalg.svds(X_train_tfidf,k=100)
print(s)

[  3.81765997   3.82091415   3.83358793   3.83683577   3.84037065
   3.85404427   3.8670715    3.87474636   3.88791474   3.89906475
   3.90668952   3.90865025   3.91221644   3.92045755   3.9377328
   3.94804621   3.95681301   3.96117128   3.98614042   3.99278686
   3.99375524   3.99969073   4.01823888   4.02302262   4.03818263
   4.05056107   4.06517333   4.07964519   4.08211187   4.09518792
   4.12256128   4.12912362   4.13937955   4.15094987   4.16790757
   4.18014174   4.20364096   4.20832978   4.23513686   4.25070119
   4.26914119   4.28770886   4.30605735   4.32106727   4.34081218
   4.35761163   4.37765991   4.39727357   4.40993431   4.42629122
   4.43354288   4.48741765   4.49913101   4.53807685   4.55039631
   4.56995463   4.58405586   4.62445108   4.63845478   4.66931913
   4.69417199   4.73801623   4.74971926   4.78219596   4.81213052
   4.82015852   4.84838011   4.86521539   4.89190857   4.89872394
   4.94012327   4.96358448   5.00348745   5.05472061   5.0823723
   5.1422803

In [16]:
rand_array = [0]*16
for i in range(1,17):    
    svd = TruncatedSVD(n_components=i, random_state=42)
    transformed_tfidf = svd.fit_transform(X_train_tfidf)
    print("Dimensions of TF-IDF vector after LSI: "+ str(transformed_tfidf.shape))
    from sklearn.metrics import confusion_matrix
    km = KMeans(n_clusters=20, init='k-means++', max_iter=300, n_init=10)
    km.fit(transformed_tfidf)
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(new_target, km.labels_))
    print("Completeness: %0.3f" % metrics.completeness_score(new_target, km.labels_))
    print("V-measure: %0.3f" % metrics.v_measure_score(new_target, km.labels_))
    print("Adjusted Rand-Index: %.3f"
          % metrics.adjusted_rand_score(new_target, km.labels_))
    print("Adjusted Mutual-Index: %.3f"
          % metrics.adjusted_mutual_info_score(new_target, km.labels_))
    #print(confusion_matrix(new_target, km.labels_))
    rand_array[i%16] = metrics.adjusted_rand_score(new_target, km.labels_)

Dimensions of TF-IDF vector after LSI: (18846, 1)
Homogeneity: 0.024
Completeness: 0.026
V-measure: 0.025
Adjusted Rand-Index: 0.005
Adjusted Mutual-Index: 0.021
Dimensions of TF-IDF vector after LSI: (18846, 2)
Homogeneity: 0.233
Completeness: 0.246
V-measure: 0.240
Adjusted Rand-Index: 0.074
Adjusted Mutual-Index: 0.231
Dimensions of TF-IDF vector after LSI: (18846, 3)
Homogeneity: 0.305
Completeness: 0.322
V-measure: 0.313
Adjusted Rand-Index: 0.109
Adjusted Mutual-Index: 0.302
Dimensions of TF-IDF vector after LSI: (18846, 4)
Homogeneity: 0.347
Completeness: 0.373
V-measure: 0.360
Adjusted Rand-Index: 0.133
Adjusted Mutual-Index: 0.345
Dimensions of TF-IDF vector after LSI: (18846, 5)
Homogeneity: 0.369
Completeness: 0.403
V-measure: 0.385
Adjusted Rand-Index: 0.152
Adjusted Mutual-Index: 0.367
Dimensions of TF-IDF vector after LSI: (18846, 6)
Homogeneity: 0.383
Completeness: 0.419
V-measure: 0.400
Adjusted Rand-Index: 0.161
Adjusted Mutual-Index: 0.381
Dimensions of TF-IDF vector 

In [17]:
print(rand_array)
print(np.amax(rand_array))
#best is n_components = 14

[0.16610018641973545, 0.005038175717161346, 0.07431684507804032, 0.10942015924589377, 0.1331020150519322, 0.15163829817892238, 0.161112443551992, 0.16461392110608353, 0.16364912251439706, 0.15392836547665198, 0.15651054627346245, 0.1630391944481179, 0.16162064398619652, 0.1771628237016831, 0.17394282296980063, 0.16672208652985535]
0.177162823702


In [18]:
rand_array = [0]*15
for i in range(1,15):
    model = NMF(n_components=i, init='random', random_state=0)
    nmf_mat= model.fit_transform(X_train_tfidf) 
    print(nmf_mat.shape)
    km = KMeans(n_clusters=20, init='k-means++', max_iter=300, n_init=10)
    km.fit(nmf_mat)
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(new_target, km.labels_))
    print("Completeness: %0.3f" % metrics.completeness_score(new_target, km.labels_))
    print("V-measure: %0.3f" % metrics.v_measure_score(new_target, km.labels_))
    print("Adjusted Rand-Index: %.3f"% metrics.adjusted_rand_score(new_target, km.labels_))
    print("Adjusted Mutual-Index: %.3f"% metrics.adjusted_mutual_info_score(new_target, km.labels_))
    #print(confusion_matrix(new_target, km.labels_))
    rand_array[i%15] = metrics.adjusted_rand_score(new_target, km.labels_)

(18846, 1)
Homogeneity: 0.024
Completeness: 0.026
V-measure: 0.025
Adjusted Rand-Index: 0.005
Adjusted Mutual-Index: 0.021
(18846, 2)
Homogeneity: 0.218
Completeness: 0.229
V-measure: 0.223
Adjusted Rand-Index: 0.069
Adjusted Mutual-Index: 0.215
(18846, 3)
Homogeneity: 0.300
Completeness: 0.321
V-measure: 0.310
Adjusted Rand-Index: 0.110
Adjusted Mutual-Index: 0.298
(18846, 4)
Homogeneity: 0.326
Completeness: 0.347
V-measure: 0.336
Adjusted Rand-Index: 0.116
Adjusted Mutual-Index: 0.324
(18846, 5)
Homogeneity: 0.344
Completeness: 0.384
V-measure: 0.363
Adjusted Rand-Index: 0.130
Adjusted Mutual-Index: 0.342
(18846, 6)
Homogeneity: 0.371
Completeness: 0.418
V-measure: 0.393
Adjusted Rand-Index: 0.140
Adjusted Mutual-Index: 0.369
(18846, 7)
Homogeneity: 0.390
Completeness: 0.440
V-measure: 0.414
Adjusted Rand-Index: 0.150
Adjusted Mutual-Index: 0.388
(18846, 8)
Homogeneity: 0.371
Completeness: 0.412
V-measure: 0.391
Adjusted Rand-Index: 0.145
Adjusted Mutual-Index: 0.369
(18846, 9)
Homog

In [19]:
print(rand_array)
print(np.amax(rand_array))
#best is n_components = 11

[0, 0.005097770080106247, 0.0688620710970322, 0.10982684906635375, 0.11562671380262218, 0.12994058766856656, 0.1396986514932432, 0.15008039963598618, 0.14494374452247719, 0.1266882079737458, 0.1486180498436126, 0.1876807054097888, 0.15232278821066145, 0.14788239656059288, 0.15335244530518402]
0.18768070541


In [24]:
#Non linear Logarithmic transformation
tfidf_transformer_log = TfidfTransformer(sublinear_tf = True)
X_train_tfidf_log = tfidf_transformer_log.fit_transform(final_counts)

In [25]:
svd = TruncatedSVD(n_components=15, random_state=42)
transformed_tfidf_log = svd.fit_transform(X_train_tfidf_log)
km = KMeans(n_clusters=20, init='k-means++', max_iter=300, n_init=10)
km.fit(transformed_tfidf_log)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(new_target, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(new_target, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(new_target, km.labels_))
print("Adjusted Rand-Index: %.3f"% metrics.adjusted_rand_score(new_target, km.labels_))
print("Adjusted Mutual-Index: %.3f"% metrics.adjusted_mutual_info_score(new_target, km.labels_))

Homogeneity: 0.439
Completeness: 0.476
V-measure: 0.456
Adjusted Rand-Index: 0.227
Adjusted Mutual-Index: 0.437


In [26]:
model = NMF(n_components=11, init='random', random_state=0)
nmf_mat_log= model.fit_transform(X_train_tfidf_log)
km = KMeans(n_clusters=20, init='k-means++', max_iter=300, n_init=10)
km.fit(nmf_mat_log)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(new_target, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(new_target, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(new_target, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(new_target, km.labels_))
print("Adjusted Mutual-Index: %.3f"
      % metrics.adjusted_mutual_info_score(new_target, km.labels_))
#print(confusion_matrix(new_target, km.labels_))

Homogeneity: 0.474
Completeness: 0.498
V-measure: 0.486
Adjusted Rand-Index: 0.252
Adjusted Mutual-Index: 0.473
